In [1]:
import xml.etree.cElementTree as etree
import string
import subprocess 
import json
import re
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import operator
from bs4 import BeautifulSoup
import requests

In [2]:
fandomDump = 'harrypotter_pages_current.xml'

In [3]:
chracters_json = open("characters.json", encoding="utf-8").read()

In [4]:
characters = json.loads(chracters_json)

In [5]:
characterSet = set(map(lambda x: x.lower(), characters))

In [6]:
namespacestr = '{http://www.mediawiki.org/xml/export-0.10/}'
namespace = {'mw': 'http://www.mediawiki.org/xml/export-0.10/'}

In [7]:
# Check if stirng does not contain any world from blacklist
def nonRestricted(string ):
    return True

In [8]:
# Substitu space for _ and cut link
def cleanLink(link):
    split = re.split(r'[|]+', link)
    return split[0]

In [9]:
def parse_Infobox(text):
    blood = re.search(r"\|blood\s*=(.*?)\n\|", text, flags=re.DOTALL)
    if blood:
        blood = blood.groups()
    else:
        blood = "None"
    house = re.search(r"\|house\s*=\s*\[\[([\w-]+)\]]", text)
    if house:
        house = house.groups()
    else:
        house = "None"
    loyalty = re.search(r"\|loyalty\s*=(.*?)\}\}", text, flags=re.DOTALL)
    if loyalty:
        loyalty = loyalty.groups()
    else:
        loyalty = "None"
    return(blood, house, loyalty)

In [23]:
def findChartersLinks(file_name):
    actorsLinkDic = {}
    character_info = {}
    for i , (event, elem) in enumerate(etree.iterparse(file_name, events=('start', 'end', 'start-ns', 'end-ns'))):
        # In case that we find tag <page> we continue
        if event == 'end' and elem.tag == namespacestr + 'page':
            # search for tag <title> inside tag <page>
            title = elem.find('.//mw:title', namespace)
            if title is not None: 
                # save text of the title
                title = title.text.lower()
                # search for tag <text> inside tag <page> 
                if title not in characterSet:
                    continue
                textelem = elem.find('.//mw:text', namespace)
                # tag text exist and is not empty and does not contain world from blacklist
                if textelem is not None and textelem.text is not None and nonRestricted(title.lower()):
                    wiki_text=textelem.text
                    text = wiki_text
                    
                    if title == "james potter ii":
                        print(title, wiki_text)
                    
                    if "#REDIRECT" in wiki_text:
                        continue
                    
                    blood, house, loyalty = parse_Infobox(wiki_text)
                    character_info[title] = {
                        "blood":blood,
                        "house":house,
                        "loyalty": loyalty
                    }
                    if loyalty == "None" and False:
                        print(text[:12000])
                    if title == "harry potter" and False:
                        print("#REDIRECT" in wiki_text)
                        print("----------------------------------------------")
                    lower_wiki_text = wiki_text.lower()
                    # article is not redirect and it is article about character
                    if not "redirect" in lower_wiki_text:
                        link_list = []
                        # find all links in article
                        for link in re.findall(r'\[\[(.*?)\]\]',wiki_text):
                            clink = cleanLink(link).lower()
                            if clink in characterSet:
                                link_list.append(clink)
                        # get metadata from article                  
                        actorsLinkDic[title]=link_list                     
                        
    return actorsLinkDic, character_info

In [24]:
#Save dictionary to json
#with open('characterLinksDic.json', 'w+') as outfile:
#    ActorLinkDic, charInfo = findChartersLinks(fandomDump)
#    json.dump(ActorLinkDic, outfile)

In [ ]:
ActorLinkDic, charInfo = findChartersLinks(fandomDump)

In [19]:
charInfo["harry potter"]

{'blood': ('[[Half-blood]]<ref name="blood">{{COS|B|17}} - Tom Riddle states that there "are strange likenesses between us, after all. Even you must have noticed. Both half-bloods".</ref><!--Please do not change this. Pure-bloods must have no Muggle grandparents. Since Harry\'s maternal grandparents were Muggles, he can only be half-blood.-->',),
 'house': ('Gryffindor',),
 'loyalty': ("*[[Potter family]]\n*[[Weasley family]]\n*[[Albus Dumbledore]]\n*[[Dumbledore's Army]]\n*[[Order of the Phoenix]]\n*[[Hogwarts School of Witchcraft and Wizardry]]\n**[[Gryffindor]]\n**[[Gryffindor Quidditch team]]\n**[[Slug Club]]\n*[[British Ministry of Magic]]\n**[[Auror Department|Auror Office]]\n**[[Department of Magical Law Enforcement]]\n*[[Society for the Promotion of Elfish Welfare|S.P.E.W.]]\n*[[Holyhead Harpies]]\n",)}

In [ ]:
# Save dictionary to json
#with open('characterInfo.json', 'w+') as outfile:
#    json.dump(charInfo, outfile)

In [ ]:
# Create graph from dictionary
with open('characterLinksDic.json', 'r') as outfile:
    characterLinksDic = json.load(outfile)

DG = nx.DiGraph()

# add nodes from dictionary
key_links = set()
for key in characterLinksDic:
    if "potter" in key:
        print("found him", key)
    key_links.add(key)
    DG.add_node(key)
    
# Add edges to graphs
for key, value in characterLinksDic.items():
    for link in value:
        if link in key_links:
            DG.add_edge(key,link)

In [ ]:
G = DG.to_undirected()

In [ ]:
plt.figure(1, figsize=(13, 13))
nx.draw_networkx(G, node_size=10, edge_color='.1', with_labels=False, width=0.9, font_color ='#ff0000', alpha=0.2)
plt.axis('off')
plt.show()

In [ ]:
len(DG.nodes)

In [ ]:
len(DG.edges)

In [ ]:
sorted(DG.degree(), key=lambda kv: kv[1], reverse=True)

1198

In [ ]:
"Harry James Potter" in characters